In [2]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI
import json
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile

/Users/Zantana/myenv/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("No OpenAI API key found. Check your .env file.")

openai.api_key = openai_api_key

In [11]:
#Instagram always stops the script after downloading the Profile pic.
class GetInstagramProfile():
    def __init__(self) -> None:
        # Initializes the Instaloader object for downloading data from Instagram.
        self.L = instaloader.Instaloader()

    def download_users_profile_picture(self, username):
        # Downloads the profile picture of the specified username.
        self.L.download_profile(username, profile_pic_only=True)

    def download_users_posts_with_periods(self, username):
        # Downloads posts of a user within a specific date range.
        posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
        SINCE = datetime(2034, 2, 1)  # Start date
        UNTIL = datetime(2024, 2, 14)  # End date

        # Filters posts within the specified date range and downloads them.
        for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
            self.L.download_post(post, username)

    def download_hastag_posts(self, hashtag):
        # Downloads posts associated with a specific hashtag.
        for post in instaloader.Hashtag.from_name(self.L.context, hashtag).get_posts():
            self.L.download_post(post, target='#'+hashtag)

    def get_users_followers(self, user_name):
        '''Note: login required to get a profile's followers.'''
        # Prompts the user to login for accessing follower data.
        self.L.login(input("testitest_97"), input("datasciencerocks"))
        profile = instaloader.Profile.from_username(self.L.context, user_name)
        file = open("follower_names.txt", "a+")
        for followee in profile.get_followers():
            username = followee.username
            file.write(username + "\n")
            print(username)

    def get_users_followings(self, user_name):
        '''Note: login required to get a profile's followings.'''
        # Similar to get_users_followers, but for retrieving who a user is following.
        self.L.login(input("testitest_97"), input("datasciencerocks"))
        profile = instaloader.Profile.from_username(self.L.context, user_name)
        file = open("following_names.txt", "a+")
        for followee in profile.get_followees():
            username = followee.username
            file.write(username + "\n")
            print(username)

    def get_post_comments(self, username):
        # Retrieves and prints comments for each post of a specified user.
        posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
        for post in posts:
            for comment in post.get_comments():
                print("comment.id  : "+str(comment.id))
                print("comment.owner.username  : "+comment.owner.username)
                print("comment.text  : "+comment.text)
                print("comment.created_at_utc  : "+str(comment.created_at_utc))
                print("************************************************")

    def get_post_info_csv(self, username):
        # Extracts detailed information about each post from a specified user and saves it into a CSV file.
        with open(username+'.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
            for post in posts:
                posturl = "https://www.instagram.com/p/"+post.shortcode
                writer.writerow(["post", post.mediaid, post.profile, post.caption, post.date, post.location, posturl, post.typename, post.mediacount, post.caption_hashtags, post.caption_mentions, post.tagged_users, post.likes, post.comments, post.title, post.url])
                
                for comment in post.get_comments():
                    writer.writerow(["comment", comment.id, comment.owner.username, comment.text, comment.created_at_utc])

if __name__ == "__main__":
    cls = GetInstagramProfile()
    # Uncomment the method you want to run. Make sure to provide the correct usernames or hashtags.
    #cls.download_users_profile_picture("zuckerjagdwurst")
    cls.download_users_posts_with_periods("zuckerjagdwurst")
    #cls.download_hastag_posts("zuckerjagdwurst")
    # cls.get_users_followers("zuckerjagdwurst")
    # cls.get_users_followings("zuckerjagdwurst")
   #cls.get_post_comments("zuckerjagdwurst")
    cls.get_post_info_csv("zuckerjagdwurst")

JSON Query to api/v1/media/3208186110936122731/comments/: HTTP error code 403. [retrying; skip with ^C]
JSON Query to api/v1/media/3208186110936122731/comments/: HTTP error code 403. [retrying; skip with ^C]


ConnectionException: JSON Query to api/v1/media/3208186110936122731/comments/: HTTP error code 403.

In [16]:
#also not working
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
 
# Launch the Chrome browser
service = Service('path/to/chromedriver')
capabilities = DesiredCapabilities.CHROME.copy()
driver = webdriver.Chrome(service=service, desired_capabilities=capabilities)
driver = webdriver.Chrome('path/to/chromedriver')
 
# Go to the Instagram post URL
post_url = 'https://www.instagram.com/p/<post_id>/'
driver.get(post_url)
 
# Log in to Instagram
username = 'your_username'
password = 'your_password'
username_field = driver.find_element_by_name('username')
password_field = driver.find_element_by_name('password')
username_field.send_keys(username)
password_field.send_keys(password)
password_field.submit()
import time
# Wait for the page to load
time.sleep(5)
# Get the comments
comments = []
comment_divs = driver.find_elements_by_xpath('//div[@class="C4VMK"]/span')
for comment_div in comment_divs:    
    comments.append(comment_div.text)


TypeError: __init__() got an unexpected keyword argument 'desired_capabilities'

In [ ]:
#fuck it we go for amazon reviews now B01N1O0F2K

In [24]:
pip install scrapy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:

pip install scrapeops-scrapy-proxy-sdk



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:

pip install scrapeops-scrapy

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 4.7 MB/s eta 0:00:00a 0:00:01
  Using cached json5-0.9.14-py2.py3-none-any.whl.metadata (10 kB)
Using cached json5-0.9.14-py2.py3-none-any.whl (19 kB)
  Created wheel for scrapeops-scrapy: filename=scrapeops_scrapy-0.5.4-py3-none-any.whl size=38280 sha256=25828b3a673e2774a4d05174c5502c33667d32ae04943c74e61d4c3c5b3a0043
  Stored in directory: /Users/Zantana/Library/Caches/pip/wheels/fd/10/ad/bda9123aff278b3db08e7a7e4611ba01e7ab7467de4512fd3c
Successfully built scrapeops-scrapy

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:

import scrapy
from urllib.parse import urljoin

class AmazonReviewsSpider(scrapy.Spider):
    name = "amazon_reviews"

    def start_requests(self):
        asin_list = ['B09G9FPHY6']
        for asin in asin_list:
            amazon_reviews_url = f'https://www.amazon.com/product-reviews/{asin}/'
            yield scrapy.Request(url=amazon_reviews_url, callback=self.parse_reviews, meta={'asin': asin})

    def parse_reviews(self, response):
        asin = response.meta['asin']
        
        ## Parse Product Reviews
        review_elements = response.css("#cm_cr-review_list div.review")
        for review_element in review_elements:
            yield {
                    "asin": asin,
                    "text": "".join(review_element.css("span[data-hook=review-body] ::text").getall()).strip(),
                    "title": review_element.css("*[data-hook=review-title]>span::text").get(),
                    "location_and_date": review_element.css("span[data-hook=review-date] ::text").get(),
                    "verified": bool(review_element.css("span[data-hook=avp-badge] ::text").get()),
                    "rating": review_element.css("*[data-hook*=review-star-rating] ::text").re(r"(\d+\.*\d*) out")[0],
                    }




ImportError: dlopen(/Users/Zantana/myenv/lib/python3.8/site-packages/lxml/etree.cpython-38-darwin.so, 0x0002): symbol not found in flat namespace '_exsltDateXpathCtxtRegister'

In [31]:
pip install lxml --force-reinstall

  Using cached lxml-5.1.0-cp38-cp38-macosx_10_9_universal2.whl.metadata (3.5 kB)
Using cached lxml-5.1.0-cp38-cp38-macosx_10_9_universal2.whl (6.2 MB)
  Attempting uninstall: lxml
    Found existing installation: lxml 5.1.0
    Uninstalling lxml-5.1.0:
      Successfully uninstalled lxml-5.1.0

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
